In [1]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from calendar import monthrange
import os, json
from rasterstats import zonal_stats
import rasterio

In [2]:
data_year = 2015
project_files = "/home/sergei/Downloads/vkr/code_base/project_data/"
fires_path = project_files + "dataset_plus_grid/"
fires_nonfires = f"dataset_{str(data_year)}.geojson"
meteo_data = f"irkutsk_{str(data_year)}_hourly.json"
meteo_data_path = project_files + "meteo_data/data/"
# aggregation period 7 days
agg_period = 7
project_crs = "epsg:3857"
print(
    "Paths:", project_files, fires_path, meteo_data_path, fires_nonfires, meteo_data,
    "Aggregation period:", agg_period, "Project crs:", project_crs, sep="\n"
)

Paths:
/home/sergei/Downloads/vkr/code_base/project_data/
/home/sergei/Downloads/vkr/code_base/project_data/dataset_plus_grid/
/home/sergei/Downloads/vkr/code_base/project_data/meteo_data/data/
dataset_2015.geojson
irkutsk_2015_hourly.json
Aggregation period:
7
Project crs:
epsg:3857


#### Connect meteo factors

In [3]:
fires_nonfires_ds = gpd.read_file(fires_path + fires_nonfires)
if not fires_nonfires_ds.crs == project_crs:
    print(f"Converting \"{fires_nonfires_ds.name}\" to {project_crs}")
    fires_nonfires_ds = fires_nonfires_ds.to_crs(project_crs)
print(fires_nonfires_ds, fires_nonfires_ds.columns, fires_nonfires_ds.dtypes, fires_nonfires_ds.event_date, sep="\n")

      year        lat         lon event_date  is_fire  grid_lat  grid_lon  \
0     2015  60.021700  115.867900 2015-07-16        1      60.0     116.0   
1     2015  59.717800  115.041000 2015-08-06        1      59.5     115.0   
2     2015  59.576100  113.984000 2015-09-25        1      59.5     114.0   
3     2015  59.487700  113.668900 2015-09-24        1      59.5     113.5   
4     2015  59.392200  114.066000 2015-07-12        1      59.5     114.0   
...    ...        ...         ...        ...      ...       ...       ...   
3079  2015  55.518063  104.196277 2015-10-21        0      55.5     104.0   
3080  2015  57.855626  115.913720 2015-10-22        0      58.0     116.0   
3081  2015  54.588979  105.621756 2015-05-12        0      54.5     105.5   
3082  2015  55.455473   99.109603 2015-04-01        0      55.5      99.0   
3083  2015  53.911422   99.010028 2015-04-30        0      54.0      99.0   

                                               geometry  
0     POLYGON ((1

In [4]:
# load meteo data json for the given
meteo_data_p = meteo_data_path + meteo_data
print("Path:", meteo_data_p)
with open(meteo_data_p, mode='r', encoding="Windows-1251") as f:
    meteo_data = json.load(f)
print(type(meteo_data))

Path: /home/sergei/Downloads/vkr/code_base/project_data/meteo_data/data/irkutsk_2015_hourly.json
<class 'dict'>


In [5]:
# json data exploration
for k, v in meteo_data.items():
    print("Keys:", k, type(k), "Values:", type(meteo_data[k]))

print(
    "Meta length:", len(meteo_data["METADATA"]),
    "Meta:", json.dumps(meteo_data["METADATA"], indent=4, ensure_ascii=False),
    "Data length:", len(meteo_data["DATA"]),
    "Data[0]:", json.dumps(meteo_data["DATA"][0], indent=4), sep="\n",
)

# print the data fields russian description
metadata_desc_rus = {}
for k, v in meteo_data["METADATA"].items():
    metadata_desc_rus[k] = v["DESC_RUS"]
print("Data fields russian description:", json.dumps(metadata_desc_rus, indent=4, ensure_ascii=False))

# data structure
# METADATA: {"factor1": {}, "factor2": {}, ...}, DATA: [{factor1: val1, factor2: val2, ...}, {factor1: val, ...}, ...]

# ensure that all metadata keys used in data keys
# only id is an extra key not stated in metadata
d = {}
for k in meteo_data["DATA"][0].keys():
    d[k] = k in meteo_data["METADATA"].keys()
print(d)

Keys: METADATA <class 'str'> Values: <class 'dict'>
Keys: DATA <class 'str'> Values: <class 'list'>
Meta length:
20
Meta:
{
    "SOILW40": {
        "MIN": 0.15,
        "DESC_RUS": "Влажность почвы (в слое 10-40см), %",
        "UNSIGNED": 1,
        "DESC_ENG": "Soil humidity (in layer 10-40cm), %",
        "MAX": 0.47
    },
    "SOILW200": {
        "MIN": 0.16,
        "DESC_RUS": "Влажность почвы (в слое 100-200см), %",
        "UNSIGNED": 1,
        "DESC_ENG": "Soil humidity (in layer 100-200cm), %",
        "MAX": 0.46
    },
    "TMIN": {
        "MIN": -55.6,
        "DESC_RUS": "Минимальная температура, °С",
        "DESC_ENG": "Minimum temperature, °С",
        "MAX": 31.6
    },
    "SNOD": {
        "MIN": 0,
        "DESC_RUS": "Глубина снега, м",
        "UNSIGNED": 1,
        "DESC_ENG": "Snow depth, m",
        "MAX": 1.585
    },
    "TMPGR40": {
        "MIN": -17.45,
        "DESC_RUS": "Температура почвы (в слое 10-40см), °С",
        "DESC_ENG": "Soil temperatur

In [6]:
meteo_ds = pd.DataFrame(meteo_data["DATA"])
# pd.options.display.max_columns = None
print(meteo_ds, meteo_ds.dtypes, sep="\n")

                                   ID  SOILW40  SOILW200  TMIN   SNOD  \
0        2015-01-01 00:00:00 114.5 61      NaN       NaN   NaN    NaN   
1          2015-01-01 00:00:00 116 56      NaN       NaN   NaN    NaN   
2        2015-01-01 00:00:00 117.5 53      NaN       NaN   NaN    NaN   
3          2015-01-01 00:00:00 111 52      NaN       NaN   NaN    NaN   
4        2015-01-01 00:00:00 105 52.5      NaN       NaN   NaN    NaN   
...                               ...      ...       ...   ...    ...   
1781671    2015-12-31 18:00:00 104 61     0.37      0.33 -15.4  0.336   
1781672     2015-12-31 18:00:00 99 61     0.37      0.34 -22.1  0.371   
1781673  2015-12-31 18:00:00 103.5 63     0.28      0.33 -22.3  0.608   
1781674  2015-12-31 18:00:00 109 59.5     0.34      0.34 -16.0  0.403   
1781675  2015-12-31 18:00:00 102 54.5     0.31      0.28 -20.1  0.307   

         TMPGR40      T        DATE  WIND_SPEED  TMPGR100  ... TMPGR10    LON  \
0            NaN    NaN  2015-01-01       

In [7]:
metadata_desc_rus

{'SOILW40': 'Влажность почвы (в слое 10-40см), %',
 'SOILW200': 'Влажность почвы (в слое 100-200см), %',
 'TMIN': 'Минимальная температура, °С',
 'SNOD': 'Глубина снега, м',
 'TMPGR40': 'Температура почвы (в слое 10-40см), °С',
 'T': 'Температура, °С',
 'DATE': 'Дата',
 'WIND_SPEED': 'Скорость ветра, м/c',
 'TMPGR100': 'Температура почвы (в слое 40-100см), °С',
 'LAT': 'Широта, °',
 'TMPGR10': 'Температура почвы (в слое 0-10см), °С',
 'LON': 'Долгота, °',
 'TMAX': 'Максимальная температура, °С',
 'RH': 'Относительная влажность, %',
 'APCP': 'Количество осадков, кг/м2',
 'TIME': 'Время',
 'WIND_DIR': 'Направление ветра, °',
 'TMPGR200': 'Температура почвы (в слое 100-200см), °С',
 'SOILW10': 'Влажность почвы (в слое 0-10см), %',
 'SOILW100': 'Влажность почвы (в слое 40-100см), %'}

In [8]:
meteo_ds["DATE"] = pd.to_datetime(meteo_ds["DATE"])
meteo_ds["LAT"] = meteo_ds["LAT"].astype(float)
meteo_ds["LON"] = meteo_ds["LON"].astype(float)
meteo_ds["LAT"].head(5), meteo_ds["LON"].head(5), meteo_ds["DATE"].head(5)

(0    61.0
 1    56.0
 2    53.0
 3    52.0
 4    52.5
 Name: LAT, dtype: float64,
 0    114.5
 1    116.0
 2    117.5
 3    111.0
 4    105.0
 Name: LON, dtype: float64,
 0   2015-01-01
 1   2015-01-01
 2   2015-01-01
 3   2015-01-01
 4   2015-01-01
 Name: DATE, dtype: datetime64[ns])

In [9]:
# convert meteo data to the gmt+8
meteo_date = meteo_ds.DATE
meteo_time = meteo_ds.TIME
meteo_date_time = pd.to_datetime(meteo_date.astype(str) + " " + meteo_time)
meteo_ds["date_time"] = meteo_date_time + pd.DateOffset(hours=8)

In [10]:
def mode_num(x: pd.Series) -> float:
    """
    Description: counts mode for the series and gets its first value
    Params:
    x - the series
    Returns: one of the series modes or numpy nan if the mode was not found
    """
    m = x.mode()
    if len(m) == 0:
        return np.nan
    return float(m[0])

ex_cols = ["ID", "DATE", "TIME", "LAT", "LON", "date_time"]
target_columns = meteo_ds.columns[~meteo_ds.columns.isin(ex_cols)]

In [11]:
# T, TMAX (?), RH, WIND_DIR, WIND_SPEED, APCP
# !!! Attention: nan and None values are skipped
# SOILW10, TMPGR10 - ?? (soil temperature and humidity at 0-10 sm). Only possible to add starting from 2006
# aggregate for the aggregation period = 7
factors = {}
for i in fires_nonfires_ds.index:
    lat = fires_nonfires_ds.iloc[i].grid_lat
    lon = fires_nonfires_ds.iloc[i].grid_lon
    date = fires_nonfires_ds.iloc[i].event_date
    start = date - pd.DateOffset(days=agg_period)
    # include the target day as well, totally there're actually 8 days for the aggregation (the last one included)
    end = date + pd.DateOffset(days=1)
    # print("Start date:", start, "End date:", end, "Grid lat:", lat, "Grid lon:", lon)
    date_mask = (meteo_ds["date_time"] >= start) & (meteo_ds["date_time"] <= end)
    meteo_date_mask = meteo_ds[date_mask]
    # print(meteo_date_mask)
    # consider the floating point error
    coord_mask = ((meteo_date_mask["LAT"] - lat).abs() <= 1.0e-5) & ((meteo_date_mask["LON"] - lon).abs() <= 1.0e-5)
    meteo_factors = meteo_date_mask[coord_mask]
    
    factors[i] = {}
    # should be competable with the pd.df.aggregate method
    agg_functions = ["mean", "max", "min", "std", mode_num, "median",]
    for col in target_columns:
        agg_d = meteo_factors[col].aggregate(agg_functions).to_dict()
        # factors[i][col] = {f"{col.lower()}_{k}" : v for k, v in agg_d.items()}
        agg_d_ = {f"{col.lower()}_{k}" : v for k, v in agg_d.items()}
        factors[i].update(agg_d_)

assert len(factors) == fires_nonfires_ds.shape[0]
k_ = json.dumps({k : factors[k] for k in list(factors.keys())[:1]}, indent=4)
print(k_)

/home/sergei/.venv_for_uni/venv_for_uni/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/sergei/.venv_for_uni/venv_for_uni/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/sergei/.venv_for_uni/venv_for_uni/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/sergei/.venv_for_uni/venv_for_uni/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/sergei/.venv_for_uni/venv_for_uni/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/sergei/.venv_for_uni/ven

{
    "0": {
        "soilw40_mean": 0.25,
        "soilw40_max": 0.25,
        "soilw40_min": 0.25,
        "soilw40_std": 0.0,
        "soilw40_mode_num": 0.25,
        "soilw40_median": 0.25,
        "soilw200_mean": 0.43,
        "soilw200_max": 0.43,
        "soilw200_min": 0.43,
        "soilw200_std": 0.0,
        "soilw200_mode_num": 0.43,
        "soilw200_median": 0.43,
        "tmin_mean": 15.140625,
        "tmin_max": 25.6,
        "tmin_min": 6.9,
        "tmin_std": 4.325830845051619,
        "tmin_mode_num": 9.4,
        "tmin_median": 14.5,
        "snod_mean": 0.0,
        "snod_max": 0.0,
        "snod_min": 0.0,
        "snod_std": 0.0,
        "snod_mode_num": 0.0,
        "snod_median": 0.0,
        "tmpgr40_mean": 6.025625,
        "tmpgr40_max": 6.84,
        "tmpgr40_min": 5.47,
        "tmpgr40_std": 0.39593855420971247,
        "tmpgr40_mode_num": 5.5,
        "tmpgr40_median": 6.015000000000001,
        "t_mean": 18.426562500000003,
        "t_max": 31.34,
 

In [12]:
meteo_factors_ds = pd.DataFrame.from_dict(factors, orient="index")
print(meteo_factors_ds.dtypes, meteo_factors_ds.shape, sep="\n")
meteo_factors_ds.head(5)

soilw40_mean         float64
soilw40_max          float64
soilw40_min          float64
soilw40_std          float64
soilw40_mode_num     float64
                      ...   
soilw100_max         float64
soilw100_min         float64
soilw100_std         float64
soilw100_mode_num    float64
soilw100_median      float64
Length: 96, dtype: object
(3084, 96)


,soilw40_mean,soilw40_max,soilw40_min,soilw40_std,soilw40_mode_num,soilw40_median,soilw200_mean,soilw200_max,soilw200_min,soilw200_std,...,soilw10_min,soilw10_std,soilw10_mode_num,soilw10_median,soilw100_mean,soilw100_max,soilw100_min,soilw100_std,soilw100_mode_num,soilw100_median
0,0.250000,0.25,0.25,0.000000,0.25,0.25,0.43,0.43,0.43,0.000000e+00,...,0.21,0.018491,0.23,0.23,0.270000,0.27,0.27,0.000000,0.27,0.27
1,0.264375,0.27,0.26,0.005040,0.26,0.26,0.44,0.44,0.44,0.000000e+00,...,0.20,0.002961,0.20,0.20,0.287187,0.29,0.28,0.004568,0.29,0.29
2,0.352500,0.36,0.34,0.005680,0.35,0.35,0.45,0.45,0.45,0.000000e+00,...,0.32,0.009832,0.33,0.33,0.330000,0.33,0.33,0.000000,0.33,0.33
3,0.346875,0.36,0.34,0.005351,0.35,0.35,0.43,0.43,0.43,0.000000e+00,...,0.31,0.008799,0.33,0.33,0.309063,0.31,0.30,0.002961,0.31,0.31
4,0.301935,0.31,0.30,0.004016,0.30,0.30,0.45,0.45,0.45,5.642875e-17,...,0.22,0.006046,0.23,0.23,0.349677,0.35,0.34,0.001796,0.35,0.35


In [13]:
# round to 6 decimal places
prec = 6
meteo_factors_ds = meteo_factors_ds.apply(lambda x: np.round(x, decimals=6))
factors_ds_nan = fires_nonfires_ds.join(meteo_factors_ds)
factors_ds_nan.dtypes

year                          int32
lat                         float64
lon                         float64
event_date           datetime64[ms]
is_fire                       int32
                          ...      
soilw100_max                float64
soilw100_min                float64
soilw100_std                float64
soilw100_mode_num           float64
soilw100_median             float64
Length: 104, dtype: object

In [14]:
# replace nans by the parameter mode calculated for the event month

nan_idx = factors_ds_nan[factors_ds_nan.isnull().any(axis=1)].index
# print(factors_ds.loc[nan_idx].geometry.geom_type)
print(nan_idx)

# for each row that has at least one nan value, extract the column of that value
# get a month of the row, get all values for the month and get mode, write to nan
factors_ds = factors_ds_nan.copy()
for idx in nan_idx:
    row = factors_ds.loc[idx]
    cols = row[row.isnull()].index.to_list()
    for col in cols:
        row_month = row.event_date.month
        m_ = factors_ds[factors_ds.event_date.dt.month == row_month][col].mode()
        m = 0 if len(m_) == 0 else m_[0]
        factors_ds.loc[idx, col] = m

# assert that no nans are left
assert factors_ds.isnull().sum().sum() == 0

Index([  82,   87,   88,   90,  250,  873,  908,  995, 1296, 1297, 1298, 1299,
       1302, 1303, 1395, 1497, 1666, 1673, 1684, 1705, 1756, 1763, 1766, 1870,
       1900, 1908, 2008, 2064, 2119, 2252, 2283, 2336, 2338, 2410, 2449, 2460,
       2577, 2651, 2877, 3057],
      dtype='int64')


In [15]:
# factors_ds.explore()

#### Connect raster factors

In [16]:
# topography
topography_dir = project_files + "topography/"
elevation_path = topography_dir + "elevation.tif"
slope_path = topography_dir + "slope.tif"
aspect_path = topography_dir + "aspect.tif"
# topography columns names
elevation = "elevation"
slope = "slope"
aspect = "aspect"

# vegetation
vegetation_dir = project_files + "vegetation/"
vegetation_types_path = vegetation_dir + "CompositeMerged.tif"
# vegetation column name
vegetation_t = "vegetation_type"

print(elevation_path, slope_path, aspect_path, vegetation_types_path, elevation, slope, aspect, vegetation_t, sep="\n")

/home/sergei/Downloads/vkr/code_base/project_data/topography/elevation.tif
/home/sergei/Downloads/vkr/code_base/project_data/topography/slope.tif
/home/sergei/Downloads/vkr/code_base/project_data/topography/aspect.tif
/home/sergei/Downloads/vkr/code_base/project_data/vegetation/CompositeMerged.tif
elevation
slope
aspect
vegetation_type


In [17]:
print("We'll use a dataset with the connected meteo factors onwards")
factors_ds

We'll use a dataset with the connected meteo factors onwards


,year,lat,lon,event_date,is_fire,grid_lat,grid_lon,geometry,soilw40_mean,soilw40_max,...,soilw10_min,soilw10_std,soilw10_mode_num,soilw10_median,soilw100_mean,soilw100_max,soilw100_min,soilw100_std,soilw100_mode_num,soilw100_median
0,2015,60.021700,115.867900,2015-07-16,1,60.0,116.0,"POLYGON ((12897309.459 8403569.034, 12897309.1...",0.250000,0.25,...,0.21,0.018491,0.23,0.23,0.270000,0.27,0.27,0.000000,0.27,0.27
1,2015,59.717800,115.041000,2015-08-06,1,59.5,115.0,"POLYGON ((12805346.404 8336204.874, 12805346.1...",0.264375,0.27,...,0.20,0.002961,0.20,0.20,0.287188,0.29,0.28,0.004568,0.29,0.29
2,2015,59.576100,113.984000,2015-09-25,1,59.5,114.0,"POLYGON ((12687257.185 8304795.856, 12687256.7...",0.352500,0.36,...,0.32,0.009832,0.33,0.33,0.330000,0.33,0.33,0.000000,0.33,0.33
3,2015,59.487700,113.668900,2015-09-24,1,59.5,113.5,"POLYGON ((12653525.898 8287187.746, 12653525.7...",0.346875,0.36,...,0.31,0.008799,0.33,0.33,0.309062,0.31,0.30,0.002961,0.31,0.31
4,2015,59.392200,114.066000,2015-07-12,1,59.5,114.0,"POLYGON ((12696710.022 8264660.54, 12696709.73...",0.301935,0.31,...,0.22,0.006046,0.23,0.23,0.349677,0.35,0.34,0.001796,0.35,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,2015,55.518063,104.196277,2015-10-21,0,55.5,104.0,POINT (11599076.501 7463067.792),0.316562,0.32,...,0.30,0.013854,0.32,0.32,0.297188,0.30,0.29,0.004568,0.30,0.30
3080,2015,57.855626,115.913720,2015-10-22,0,58.0,116.0,POINT (12903456.34 7937050.102),0.359375,0.36,...,0.46,0.000000,0.46,0.46,0.344688,0.35,0.34,0.005070,0.34,0.34
3081,2015,54.588979,105.621756,2015-05-12,0,54.5,105.5,POINT (11757760.078 7282500.549),0.370000,0.37,...,0.25,0.027837,0.32,0.30,0.260000,0.26,0.26,0.000000,0.26,0.26
3082,2015,55.455473,99.109603,2015-04-01,0,55.5,99.0,POINT (11032830.526 7450770.729),0.449375,0.46,...,0.36,0.026333,0.37,0.37,0.328750,0.33,0.32,0.003360,0.33,0.33


##### Connect raster factors functions

In [18]:
def get_geometry_by_geom_type(
    geometry_column: gpd.GeoSeries, geom_type: str, raster_crs: rasterio.crs.CRS
) -> tuple[gpd.GeoSeries, gpd.GeoSeries]:
    """
    Description: extract the geometry data with a given type and in a given crs
    Params:
    geometry_column - a column that contains the GeoDataFrame geometry
    geom_type - a geometry type to be extracted
    raster_crs - a crs of a raster layer
    Returns: the GeoSeries that contains the geometry column with the given type and in the given crs and a mask according
    to which this series was extracted from the original column
    """
    # print(type(geometry_column))
    geom_mask = geometry_column.geometry.geom_type == geom_type
    masked = geometry_column[geom_mask].to_crs(raster_crs).geometry
    return masked, geom_mask

In [19]:
def add_raster_factor(factor: str, factor_path: str, 
                      dataset: gpd.GeoDataFrame, copy: bool = True, 
                      statistics: list=["mean"], add_stats: dict | None = None,
                     ) -> gpd.GeoDataFrame:
    """
    Description: extracts values from the raster file describing the factor in a given vector geometry dataset
    Params:
    factor - a factor to extract
    factor_path - a path to the factor raster file 
    dataset - a GeoDataFrame with the geometry column
    copy - copy the dataset
    statistics - the statistics to apply to polygons, default: mean, should be a list
    add_stats - is an optional argument that will be passed to the zonal statistics to calculate the custom statistics
    (for ex, get the array of pixel values that intersect the given geometry). The argument should be passed in the
    following form: {'statistics_name':function_name}. Only for print use, if not none not printed
    Returns: the dataset with the factor which values are extracted from the corresponding raster file
    Limitations: works only with the first raster band and computes zonal statistics only for the first 
    value in the statistics list
    """
    # Algorithm
    # add a factor column to the dataset
    # open the raster file
    # get the raster nodata value 
    # get all available geometry types in the dataset
    # for each geom type: 
    # separate points and polygons using get_geometry_by_geom_type function
    # if type==point, perform sampling using rasterio, if type==(multi)polygon apply zonal statistics from rasterstats
    # write values to the factor column
    # endfor
    # close file
    if copy:
        dataset = dataset.copy()
    
    dataset[factor] = 0.0
    raster_band = 1
    
    factor_raster = rasterio.open(factor_path)
    factor_raster_ds = factor_raster.read(raster_band)
    # nodatavals returns the nodata value for each band, here we get only for the first one
    nodata_value = factor_raster.nodatavals[raster_band - 1]

    factor_raster_ds = factor_raster_ds.astype("float64")
  
    geometry_column = dataset.geometry
    geom_types = geometry_column.geometry.geom_type.unique()

    supported_geom_types = ["Point", "Polygon", "MultiPolygon"]
    for geom_type in geom_types:        
        geometry_by_type, geometry_by_type_mask = get_geometry_by_geom_type(geometry_column, geom_type, factor_raster.crs)
        if geom_type == supported_geom_types[0]:
            coord_list = [(x, y) for x, y in zip(geometry_by_type.x, geometry_by_type.y)]
            factor_l = [x[0] for x in factor_raster.sample(coord_list, masked=True)]
            # to get rid of the masked values by converting them to nan
            factor_arr = np.array(factor_l).astype(float)  
            dataset.loc[geometry_by_type_mask, factor] = factor_arr     # assign values
            assert dataset[geometry_by_type_mask].geometry.geom_type.unique() == [geom_type]
        elif geom_type in supported_geom_types[1:]:
            transform = factor_raster.transform
            # source: https://pythonhosted.org/rasterstats/rasterstats.html
            # params: polygons geopandas dataframe (or path to the shp file), an array from a raster image, a coordinates
            # transformation matrix and the stats to count. all_touched is used to include pixels that're touched by
            # the geometry (by default if the geometry center doesn't intersect with the pixel that doesn't count) and
            # nodata_value is used to exclude nodata value from computation
            # pass only the geometry column as the function works a little bit faster
            # print("Statistics:", statistics)
            stats = zonal_stats(
                geometry_by_type, factor_raster_ds, 
                affine=transform, stats=statistics,
                all_touched=True, nodata=nodata_value, add_stats=add_stats,  #raster_out=False,
            )
            if not add_stats is None:
                print(stats)
            poly_stats_l = [x[statistics[0]] for x in stats]
            # to get rid of the "masked" values
            poly_stats_arr = np.array(poly_stats_l).astype(float)
            dataset.loc[geometry_by_type_mask, factor] = poly_stats_arr
            assert dataset[geometry_by_type_mask].geometry.geom_type.unique() == [geom_type]
    
    if not factor_raster.closed:
        factor_raster.close()
    
    return dataset

In [20]:
def my_stat(x):
    """
    Pass as an argument "add_stats" to the add_raster_factor function to get the underlying array of pixel values 
    that intersect the polygons
    """
    return np.ma.getdata(x)

In [21]:
statistics_ = ["mean", "majority"]
print(statistics_[1])
raster_ds = add_raster_factor(elevation, elevation_path, factors_ds)
add_raster_factor(slope, slope_path, raster_ds, copy=False)
# add_raster_factor(aspect, aspect_path, raster_ds, copy=False)
add_raster_factor(aspect, aspect_path, raster_ds, copy=False, statistics=[statistics_[1]])
add_raster_factor(vegetation_t, vegetation_types_path, raster_ds, copy=False, statistics=[statistics_[1]])
print(raster_ds)

majority


/tmp/ipykernel_8618/146860073.py:54: UserWarning: Warning: converting a masked element to nan.
  factor_arr = np.array(factor_l).astype(float)
/tmp/ipykernel_8618/146860073.py:54: UserWarning: Warning: converting a masked element to nan.
  factor_arr = np.array(factor_l).astype(float)
/tmp/ipykernel_8618/146860073.py:54: UserWarning: Warning: converting a masked element to nan.
  factor_arr = np.array(factor_l).astype(float)


      year        lat         lon event_date  is_fire  grid_lat  grid_lon  \
0     2015  60.021700  115.867900 2015-07-16        1      60.0     116.0   
1     2015  59.717800  115.041000 2015-08-06        1      59.5     115.0   
2     2015  59.576100  113.984000 2015-09-25        1      59.5     114.0   
3     2015  59.487700  113.668900 2015-09-24        1      59.5     113.5   
4     2015  59.392200  114.066000 2015-07-12        1      59.5     114.0   
...    ...        ...         ...        ...      ...       ...       ...   
3079  2015  55.518063  104.196277 2015-10-21        0      55.5     104.0   
3080  2015  57.855626  115.913720 2015-10-22        0      58.0     116.0   
3081  2015  54.588979  105.621756 2015-05-12        0      54.5     105.5   
3082  2015  55.455473   99.109603 2015-04-01        0      55.5      99.0   
3083  2015  53.911422   99.010028 2015-04-30        0      54.0      99.0   

                                               geometry  soilw40_mean  \
0 

/tmp/ipykernel_8618/146860073.py:54: UserWarning: Warning: converting a masked element to nan.
  factor_arr = np.array(factor_l).astype(float)


In [22]:
e = elevation
s = slope
asp = aspect
vt = vegetation_t
raster_ds[[elevation, slope, aspect, vegetation_t]].describe()

,elevation,slope,aspect,vegetation_type
count,3082.000000,3082.000000,3039.000000,3065.000000
mean,588.395172,2.640860,129.085224,12.823165
std,270.006144,2.693616,107.459106,5.041607
min,166.000000,0.000000,0.000000,1.000000
25%,420.000000,0.950471,31.421106,11.000000
50%,516.603529,1.715637,104.516701,12.000000
75%,691.128906,3.351620,216.656303,18.000000
max,2286.000000,24.151606,359.501770,25.000000


#####  None type values fill

In [23]:
# nan masks
el_nan_m = raster_ds.elevation.isnull()
slope_nan_m = raster_ds.slope.isnull()
asp_nan_m = raster_ds.aspect.isnull()
vt_nan_m = raster_ds.vegetation_type.isnull()
print(
    "The nans number for each raster factor:",
    raster_ds[el_nan_m].shape, raster_ds[slope_nan_m].shape, raster_ds[asp_nan_m].shape, raster_ds[vt_nan_m].shape
)
print(
    "Nans in the raster factors",
    raster_ds[el_nan_m],
    raster_ds[slope_nan_m],
    raster_ds[asp_nan_m],
    raster_ds[vt_nan_m],
    sep="\n"
)

The nans number for each raster factor: (2, 108) (2, 108) (45, 108) (19, 108)
Nans in the raster factors
      year        lat         lon event_date  is_fire  grid_lat  grid_lon  \
1398  2015  64.031730  106.665120 2015-04-11        0      64.0     106.5   
2230  2015  56.839276  112.706826 2015-05-22        0      57.0     112.5   

                              geometry  soilw40_mean  soilw40_max  ...  \
1398  POINT (11873906.833 9357826.242)        0.3400         0.34  ...   
2230    POINT (12546466.47 7727338.74)        0.4125         0.42  ...   

      soilw100_mean  soilw100_max  soilw100_min  soilw100_std  \
1398           0.24          0.24          0.24           0.0   
2230           0.34          0.34          0.34           0.0   

      soilw100_mode_num  soilw100_median  elevation  slope  aspect  \
1398               0.24             0.24        NaN    NaN     NaN   
2230               0.34             0.34        NaN    NaN     NaN   

      vegetation_type  
1398     

In [24]:
# first drop the vegetation type where nan
vt_nan_idx = raster_ds[vt_nan_m].index
print(vt_nan_idx)
raster_ds_clean = raster_ds.drop(index=vt_nan_idx).reset_index(drop=True)
assert abs(raster_ds_clean.shape[0] - raster_ds.shape[0]) == vt_nan_idx.shape[0]
raster_ds_clean[raster_ds_clean.vegetation_type.isnull()].shape
# raster_ds_clean.head(2), raster_ds_clean.index
# raster_ds_clean.is_fire.value_counts()

Index([1398, 1407, 1412, 1466, 1530, 1570, 1759, 1775, 1804, 1825, 1832, 1837,
       2336, 2384, 2623, 2638, 2920, 2957, 3004],
      dtype='int64')


(0, 108)

In [25]:
# second: fill elevation and slope nans with the mean and aspect
# with the mode value for each column
# !!! Attention: changes the original dataset
prec = 6
el_nan_idx = raster_ds_clean[raster_ds_clean.elevation.isnull()].index
raster_ds_clean.loc[el_nan_idx, elevation] = raster_ds_clean.elevation.mean().round(prec)

slope_nan_idx = raster_ds_clean[raster_ds_clean.slope.isnull()].index
raster_ds_clean.loc[slope_nan_idx, slope] = raster_ds_clean.slope.mean().round(prec)

asp_nan_idx = raster_ds_clean[raster_ds_clean.aspect.isnull()].index
raster_ds_clean.loc[asp_nan_idx, aspect] = raster_ds_clean.aspect.mode()[0].round(prec)

# print(el_nan_idx, slope_nan_idx, asp_nan_idx, sep="\n")
# print(raster_ds_clean.loc[el_nan_idx, elevation], raster_ds_clean.loc[slope_nan_idx, slope],
#       raster_ds_clean.loc[asp_nan_idx, aspect], sep="\n")

In [26]:
# finally check all columns for nones/nans
print("None values left in the dataset:", raster_ds_clean.isnull().sum().sum())

None values left in the dataset: 0


#### Connect social factors

In [27]:
# social
social_dir = project_files + "social/"
roads_path = social_dir + "auto_roads.geojson"
rivers_path = social_dir + "rivers.geojson"
localities_path = social_dir + "localities_Irk_obl.geojson"
techno_objects_path = social_dir + "techno_obj.csv"
# social columns names
# roads = "elevation"
print(roads_path, rivers_path, localities_path, techno_objects_path, sep="\n")

/home/sergei/Downloads/vkr/code_base/project_data/social/auto_roads.geojson
/home/sergei/Downloads/vkr/code_base/project_data/social/rivers.geojson
/home/sergei/Downloads/vkr/code_base/project_data/social/localities_Irk_obl.geojson
/home/sergei/Downloads/vkr/code_base/project_data/social/techno_obj.csv


In [28]:
print("We'll use a dataset with the connected meteo and raster factors onwards")
raster_ds_clean.head(5)

We'll use a dataset with the connected meteo and raster factors onwards


,year,lat,lon,event_date,is_fire,grid_lat,grid_lon,geometry,soilw40_mean,soilw40_max,...,soilw100_mean,soilw100_max,soilw100_min,soilw100_std,soilw100_mode_num,soilw100_median,elevation,slope,aspect,vegetation_type
0,2015,60.0217,115.8679,2015-07-16,1,60.0,116.0,"POLYGON ((12897309.459 8403569.034, 12897309.1...",0.250000,0.25,...,0.270000,0.27,0.27,0.000000,0.27,0.27,756.285714,3.867853,171.964294,21.0
1,2015,59.7178,115.0410,2015-08-06,1,59.5,115.0,"POLYGON ((12805346.404 8336204.874, 12805346.1...",0.264375,0.27,...,0.287188,0.29,0.28,0.004568,0.29,0.29,794.625000,4.589434,194.938690,1.0
2,2015,59.5761,113.9840,2015-09-25,1,59.5,114.0,"POLYGON ((12687257.185 8304795.856, 12687256.7...",0.352500,0.36,...,0.330000,0.33,0.33,0.000000,0.33,0.33,511.181818,7.200536,15.219467,11.0
3,2015,59.4877,113.6689,2015-09-24,1,59.5,113.5,"POLYGON ((12653525.898 8287187.746, 12653525.7...",0.346875,0.36,...,0.309062,0.31,0.30,0.002961,0.31,0.31,512.181818,2.837777,7.961754,15.0
4,2015,59.3922,114.0660,2015-07-12,1,59.5,114.0,"POLYGON ((12696710.022 8264660.54, 12696709.73...",0.301935,0.31,...,0.349677,0.35,0.34,0.001796,0.35,0.35,899.625000,5.854614,201.392883,1.0


In [29]:
# 
# # tried to make a function here
# def add_social_factor(dataset: gpd.GeoDataFrame, factor_name: str, 
#                       keep_columns: list[str] | None = None, ) -> gpd.GeoDataFrame:
#     # select columns we need
#     # keep_cols = ["type", "id"], "geometry"]
#     keep_cols = []
#     keep_cols = keep_columns + [dataset.geometry.name]
#     print(keep_cols, id(keep_cols) == id(keep_columns))
#     roads_ = roads[keep_cols]
#     roads_.rename(columns={"type" : "road_type", "id" : "road_id"}, inplace=True)
    
#     # # join datasets by distance
#     # roads_ds = gpd.sjoin_nearest(raster_ds_clean, roads_, how="left", distance_col="road_dist")
#     # roads_ds.drop(columns=["index_right"], inplace=True)
#     # print(roads_ds.columns, roads_ds.shape, raster_ds_clean.shape, sep="\n")
    
#     # # drop duplicates in the index
#     # print("Duplicates indeces: ", roads_ds[roads_ds.index.duplicated()].index)
#     # social_fac_ds = roads_ds[~roads_ds.index.duplicated(keep="first")]    # keep only the first found duplicated row
#     # print("Duplicates left:", social_fac_ds2.index.duplicated().sum())
#     # social_fac_ds.reset_index(drop=True, inplace=True)
#     # social_fac_ds.index
# keep_cols = ["type", "id"]
# add_social_factor(dataset=raster_ds_clean, factor_name="road", keep_columns=keep_cols)
# keep_cols
# 

In [30]:
# write the dataset with the connected social factors to the social_fac_ds
social_fac_ds = None

In [31]:
# connect roads
roads = gpd.read_file(roads_path)
if not roads.crs == project_crs:
    print(f"Converting \"{type(roads)}\" to {project_crs}")
    roads = roads.to_crs(project_crs)
print(roads.head(5), roads.dtypes, roads.shape, roads.crs, sep="\n")

# select columns we need
keep_cols = ["type", "id", "geometry"]
roads_ = roads[keep_cols]
roads_.rename(columns={"type" : "road_type", "id" : "road_id"}, inplace=True)

# join datasets by distance
roads_ds = gpd.sjoin_nearest(raster_ds_clean, roads_, how="left", distance_col="road_dist")
roads_ds.drop(columns=["index_right"], inplace=True)
print(roads_ds.columns, roads_ds.shape, raster_ds_clean.shape, sep="\n")

# drop duplicates in the index
print("Duplicates indeces: ", roads_ds[roads_ds.index.duplicated()].index)
roads_ds = roads_ds[~roads_ds.index.duplicated(keep="first")]    # keep only the first found duplicated row
print("Duplicates left:", roads_ds.index.duplicated().sum())
roads_ds.reset_index(drop=True, inplace=True)
roads_ds.index, roads_ds

Converting "<class 'geopandas.geodataframe.GeoDataFrame'>" to epsg:3857
          type   id is_deleted symbol                created_by edited_by  \
0  федеральная  122          f   None  57cd206ea47f4dd00900000f      None   
1       прочая  123          f   None  57cd206ea47f4dd00900000f      None   
2       прочая  124          f   None  57cd206ea47f4dd00900000f      None   
3  федеральная  125          f   None  57cd206ea47f4dd00900000f      None   
4  федеральная  126          f   None  57cd206ea47f4dd00900000f      None   

                edited_on                       created_on published  \
0 2017-04-18 18:15:04.311 2017-04-18 18:15:04.311000+09:00         f   
1 2017-04-18 18:15:04.574 2017-04-18 18:15:04.574000+09:00         f   
2 2017-04-18 18:15:04.576 2017-04-18 18:15:04.576000+09:00         f   
3 2017-04-18 18:15:04.577 2017-04-18 18:15:04.577000+09:00         f   
4 2017-04-18 18:15:04.579 2017-04-18 18:15:04.579000+09:00         f   

                                

/tmp/ipykernel_8618/1040085769.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roads_.rename(columns={"type" : "road_type", "id" : "road_id"}, inplace=True)


(RangeIndex(start=0, stop=3065, step=1),
       year        lat         lon event_date  is_fire  grid_lat  grid_lon  \
 0     2015  60.021700  115.867900 2015-07-16        1      60.0     116.0   
 1     2015  59.717800  115.041000 2015-08-06        1      59.5     115.0   
 2     2015  59.576100  113.984000 2015-09-25        1      59.5     114.0   
 3     2015  59.487700  113.668900 2015-09-24        1      59.5     113.5   
 4     2015  59.392200  114.066000 2015-07-12        1      59.5     114.0   
 ...    ...        ...         ...        ...      ...       ...       ...   
 3060  2015  55.518063  104.196277 2015-10-21        0      55.5     104.0   
 3061  2015  57.855626  115.913720 2015-10-22        0      58.0     116.0   
 3062  2015  54.588979  105.621756 2015-05-12        0      54.5     105.5   
 3063  2015  55.455473   99.109603 2015-04-01        0      55.5      99.0   
 3064  2015  53.911422   99.010028 2015-04-30        0      54.0      99.0   
 
                     

In [32]:
# connect rivers
rivers = gpd.read_file(rivers_path)
if not rivers.crs == project_crs:
    print(f"Converting \"{type(rivers)}\" to {project_crs}")
    rivers = rivers.to_crs(project_crs)
print(rivers.head(5), rivers.dtypes, rivers.shape, rivers.crs, sep="\n")

# select columns we need
keep_cols = ["id", "geometry"]
rivers_ = rivers[keep_cols]
rivers_.rename(columns={"id" : "river_id"}, inplace=True)

# join datasets by distance (use dataset with the connected roads)
rivers_ds = gpd.sjoin_nearest(roads_ds, rivers_, how="left", distance_col="river_dist")
rivers_ds.drop(columns=["index_right"], inplace=True)
print(rivers_ds.columns, rivers_ds.shape, raster_ds_clean.shape, sep="\n")

# drop duplicates in the index
print("Duplicates indeces: ", rivers_ds[rivers_ds.index.duplicated()].index)
rivers_ds = rivers_ds[~rivers_ds.index.duplicated(keep="first")]    # keep only the first found duplicated row
print("Duplicates left:", rivers_ds.index.duplicated().sum())
rivers_ds.reset_index(drop=True, inplace=True)
rivers_ds.index, rivers_ds

Converting "<class 'geopandas.geodataframe.GeoDataFrame'>" to epsg:3857
              name   basin         basin2  linewidth   sq  id is_deleted  \
0         Артюгина  Енисей       Артюгина        0.3  A01   1          f   
1    Верх. Сарчиха  Енисей  Верх. Сарчиха        0.3  A01   2          f   
2  Каменный Дубчес  Енисей         Дубчес        0.3  A01   3          f   
3           Елогуй  Енисей         Елогуй        0.3  A01   8          f   
4          Делтула  Енисей          Бахта        0.3  A02  19          f   

                 created_by edited_by               edited_on  \
0  50f7a1d80d58140037000006      None 2016-05-27 16:52:38.553   
1  50f7a1d80d58140037000006      None 2016-05-27 16:52:40.242   
2  50f7a1d80d58140037000006      None 2016-05-27 16:52:40.272   
3  50f7a1d80d58140037000006      None 2016-05-27 16:52:40.284   
4  50f7a1d80d58140037000006      None 2016-05-27 16:52:40.305   

                        created_on published  \
0 2016-05-27 16:52:38.553000+08:

/tmp/ipykernel_8618/1904574751.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rivers_.rename(columns={"id" : "river_id"}, inplace=True)


(RangeIndex(start=0, stop=3065, step=1),
       year        lat         lon event_date  is_fire  grid_lat  grid_lon  \
 0     2015  60.021700  115.867900 2015-07-16        1      60.0     116.0   
 1     2015  59.717800  115.041000 2015-08-06        1      59.5     115.0   
 2     2015  59.576100  113.984000 2015-09-25        1      59.5     114.0   
 3     2015  59.487700  113.668900 2015-09-24        1      59.5     113.5   
 4     2015  59.392200  114.066000 2015-07-12        1      59.5     114.0   
 ...    ...        ...         ...        ...      ...       ...       ...   
 3060  2015  55.518063  104.196277 2015-10-21        0      55.5     104.0   
 3061  2015  57.855626  115.913720 2015-10-22        0      58.0     116.0   
 3062  2015  54.588979  105.621756 2015-05-12        0      54.5     105.5   
 3063  2015  55.455473   99.109603 2015-04-01        0      55.5      99.0   
 3064  2015  53.911422   99.010028 2015-04-30        0      54.0      99.0   
 
                     

In [33]:
# connect localities
locs = gpd.read_file(localities_path)
if not locs.crs == project_crs:
    print(f"Converting \"{type(locs)}\" to {project_crs}")
    locs = locs.to_crs(project_crs)
print(locs.head(5), locs.columns, locs.dtypes, locs.shape, locs.crs, sep="\n")

# select columns we need
keep_cols = ["type", "id", "geometry"]
locs_ = locs[keep_cols]
locs_.rename(columns={"type" : "locality_type", "id" : "locality_id"}, inplace=True)

# join datasets by distance (use rivers_ds)
locs_ds = gpd.sjoin_nearest(rivers_ds, locs_, how="left", distance_col="locality_dist")
locs_ds.drop(columns=["index_right"], inplace=True)
print(locs_ds.columns, locs_ds.shape, raster_ds_clean.shape, sep="\n")

# drop duplicates in the index
print("Duplicates indeces: ", locs_ds[locs_ds.index.duplicated()].index)
locs_ds = locs_ds[~locs_ds.index.duplicated(keep="first")]    # keep only the first found duplicated row
print("Duplicates left:", locs_ds.index.duplicated().sum())
locs_ds.reset_index(drop=True, inplace=True)
locs_ds.index, locs_ds.head(2)

Converting "<class 'geopandas.geodataframe.GeoDataFrame'>" to epsg:3857
            name              type                                    name_MO  \
0          Аларь     село сельский          Муниципальное образование «Аларь»   
1  Александровск     село сельский  Муниципальное образование «Александровск»   
2        Алзобей  деревня сельский          Муниципальное образование «Аларь»   
3          Аляты     село сельский          Муниципальное образование «Аляты»   
4      Ангарский  поселок сельский      Муниципальное образование «Ангарский»   

          code distance       ado      id  \
0  25123902001       44  Аларский  01.���   
1  25123904001       10  Аларский  01.���   
2  25123902002       36  Аларский  01.���   
3  25123907001       55  Аларский  01.���   
4  25123910001       55  Аларский  01.���   

                                           query  \
0          Аларь село Аларский иркутская область   
1  Александровск село Аларский иркутская область   
2     Алзобей 

/tmp/ipykernel_8618/4180143233.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locs_.rename(columns={"type" : "locality_type", "id" : "locality_id"}, inplace=True)


(RangeIndex(start=0, stop=3065, step=1),
    year      lat       lon event_date  is_fire  grid_lat  grid_lon  \
 0  2015  60.0217  115.8679 2015-07-16        1      60.0     116.0   
 1  2015  59.7178  115.0410 2015-08-06        1      59.5     115.0   
 
                                             geometry  soilw40_mean  \
 0  POLYGON ((12897309.459 8403569.034, 12897309.1...      0.250000   
 1  POLYGON ((12805346.404 8336204.874, 12805346.1...      0.264375   
 
    soilw40_max  ...      aspect  vegetation_type  road_type  road_id  \
 0         0.25  ...  171.964294             21.0     прочая      152   
 1         0.27  ...  194.938690              1.0     прочая      152   
 
        road_dist  river_id    river_dist  locality_type  locality_id  \
 0  193759.177102       228  21785.981741  село сельский       05.���   
 1  231036.127592       219  23197.616858  село сельский       05.���   
 
    locality_dist  
 0  114105.573893  
 1  219916.158194  
 
 [2 rows x 116 columns])

In [34]:
techno_obj_raw = gpd.pd.read_csv(techno_objects_path, sep=";", usecols=["id", "WKT"])
techno_obj_geom = gpd.GeoSeries.from_wkt(techno_obj_raw["WKT"])
techno_obj = gpd.GeoDataFrame(data=techno_obj_raw, geometry=techno_obj_geom, crs="epsg:4326")
if not techno_obj.crs == project_crs:
    print(f"Converting \"{type(techno_obj)}\" to {project_crs}")
    techno_obj = techno_obj.to_crs(project_crs)
print(techno_obj.head(5), techno_obj.columns, techno_obj.dtypes, techno_obj.shape, techno_obj.crs, sep="\n")

# select columns we need
keep_cols = ["geometry"]
techno_obj_ = techno_obj[keep_cols]
# techno_obj_.rename(columns={"id" : "techno_obj_id"}, inplace=True)
# print(techno_obj_)

# join datasets by distance (use locs_ds)
techno_obj_ds = gpd.sjoin_nearest(locs_ds, techno_obj_, how="left", distance_col="techno_obj_dist")
techno_obj_ds.drop(columns=["index_right"], inplace=True)
print(techno_obj_ds.columns, techno_obj_ds.shape, raster_ds_clean.shape, sep="\n")
# print(techno_obj_ds.head(5))
# techno_obj_ds[techno_obj_ds.techno_obj_dist.isnull() != True]

# drop duplicates in the index
print("Duplicates indeces: ", techno_obj_ds[techno_obj_ds.index.duplicated()].index)
social_fac_ds = techno_obj_ds[~techno_obj_ds.index.duplicated(keep="first")]    # keep only the first found duplicated row
print("Duplicates left:", social_fac_ds.index.duplicated().sum())
social_fac_ds.reset_index(drop=True, inplace=True)
social_fac_ds.index

Converting "<class 'geopandas.geodataframe.GeoDataFrame'>" to epsg:3857
   id                                                WKT  \
0   0  MULTIPOLYGON (((105.999880426311 56.7999540577...   
1   1  MULTIPOLYGON (((105.985713935395 56.8240593653...   
2   2  MULTIPOLYGON (((105.967972910118 56.8666326931...   
3   3  MULTIPOLYGON (((105.974120505029 56.8693588037...   
4   4  MULTIPOLYGON (((105.98247195472 56.85509207094...   

                                            geometry  
0  MULTIPOLYGON (((11799852.713 7719340.458, 1179...  
1  MULTIPOLYGON (((11798275.707 7724242.632, 1179...  
2  MULTIPOLYGON (((11796300.785 7732908.265, 1179...  
3  MULTIPOLYGON (((11796985.132 7733463.49, 11796...  
4  MULTIPOLYGON (((11797914.811 7730558.243, 1179...  
Index(['id', 'WKT', 'geometry'], dtype='object')
id             int64
WKT           object
geometry    geometry
dtype: object
(1809, 3)
epsg:3857
Index(['year', 'lat', 'lon', 'event_date', 'is_fire', 'grid_lat', 'grid_lon',
       'geome

RangeIndex(start=0, stop=3065, step=1)

In [35]:
print(social_fac_ds.dtypes, social_fac_ds.shape, social_fac_ds.crs, sep="\n")
social_fac_ds.head(5)

year                        int32
lat                       float64
lon                       float64
event_date         datetime64[ms]
is_fire                     int32
                        ...      
river_dist                float64
locality_type              object
locality_id                object
locality_dist             float64
techno_obj_dist           float64
Length: 117, dtype: object
(3065, 117)
EPSG:3857


,year,lat,lon,event_date,is_fire,grid_lat,grid_lon,geometry,soilw40_mean,soilw40_max,...,vegetation_type,road_type,road_id,road_dist,river_id,river_dist,locality_type,locality_id,locality_dist,techno_obj_dist
0,2015,60.0217,115.8679,2015-07-16,1,60.0,116.0,"POLYGON ((12897309.459 8403569.034, 12897309.1...",0.250000,0.25,...,21.0,прочая,152,193759.177102,228,21785.981741,село сельский,05.���,114105.573893,493120.164862
1,2015,59.7178,115.0410,2015-08-06,1,59.5,115.0,"POLYGON ((12805346.404 8336204.874, 12805346.1...",0.264375,0.27,...,1.0,прочая,152,231036.127592,219,23197.616858,село сельский,05.���,219916.158194,384503.700491
2,2015,59.5761,113.9840,2015-09-25,1,59.5,114.0,"POLYGON ((12687257.185 8304795.856, 12687256.7...",0.352500,0.36,...,11.0,прочая,152,281283.822800,220,1721.018231,село сельский,17.���,182717.515536,295530.312784
3,2015,59.4877,113.6689,2015-09-24,1,59.5,113.5,"POLYGON ((12653525.898 8287187.746, 12653525.7...",0.346875,0.36,...,15.0,прочая,152,312789.373320,220,25686.184906,село сельский,17.���,142391.828552,263925.972009
4,2015,59.3922,114.0660,2015-07-12,1,59.5,114.0,"POLYGON ((12696710.022 8264660.54, 12696709.73...",0.301935,0.31,...,1.0,прочая,152,316608.671509,220,15611.137720,поселок сельский,05.���,158680.082953,264187.329925


In [36]:
# drop rows where the distance to the locality < 50 m or to the techno object < 100 m
# then drop techno_obj_dist colum
loc_col = "locality_dist"
tech_col = "techno_obj_dist"
idx_ = social_fac_ds[(social_fac_ds[loc_col] < 50) | (social_fac_ds[tech_col] < 100)].index
print(social_fac_ds[(social_fac_ds[loc_col] < 50)].shape, social_fac_ds[(social_fac_ds[tech_col] < 100)].shape)
s_ds = social_fac_ds.drop(index=idx_)

soc_fac_ds = s_ds.reset_index(drop=True)
print(soc_fac_ds.shape, social_fac_ds.shape)
print(
    "Distance to locality < 50:",soc_fac_ds[soc_fac_ds[loc_col] < 50].shape, 
    "Distance to techno obj < 100:", soc_fac_ds[soc_fac_ds[tech_col] < 100].shape,
    sep="\n"
)
print("Duplicates in idx:", soc_fac_ds.index.duplicated().sum())
vc = soc_fac_ds.is_fire.value_counts()
print("Fires number:", vc[1], "Non-fires number:", vc[0])

print("Drop techno_obj_dist column")
soc_fac_ds.drop(columns=["techno_obj_dist"], inplace=True)
print("Dropped?", not soc_fac_ds.columns.isin(["techno_obj_dist"]).any())
print("Shape:", soc_fac_ds.shape)

(278, 117) (109, 117)
(2751, 117) (3065, 117)
Distance to locality < 50:
(0, 117)
Distance to techno obj < 100:
(0, 117)
Duplicates in idx: 0
Fires number: 1093 Non-fires number: 1658
Drop techno_obj_dist column
Dropped? True
Shape: (2751, 116)


In [37]:
# m = soc_fac_ds.loc[:, ["lat", "lon", "geometry"]].explore(color="red")
m = soc_fac_ds.explore(color="red")
rivers_.explore(m=m, color="blue")
roads_.explore(m=m, color="black")
locs_.explore(m=m, color="green")
techno_obj_.explore(m=m, color="purple")

# save the interactive map
maps_dir = "interactive_maps" + "/fires_geo_factors_maps"
if not os.path.exists(maps_dir):
    os.mkdir(maps_dir)
m.save(maps_dir + f"/fires_geo_factors_map_{data_year}.html")

In [38]:
# write file
write_path = project_files + "fires_with_factors/"
if not os.path.exists(write_path):
  os.mkdir(write_path)
soc_fac_ds.to_file(write_path + "factor_dataset_" + str(data_year) + ".geojson")